In [ ]:
import ee
# import folium
from shapely.geometry import Point
import numpy as np
import functools
import operator
import time
import random

import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

!pip install -q geopandas
import geopandas as gpd

# NEED TO CLEAN NOTEBOOK!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Gy-uLBcOYUKv786R8YEshrhGAXMU4YSEuczhgYVcDOY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh0TplN9_KXkvToNzmbm6-UW-AzMzs1_c2Sgm6O0onEXQLWseQ-SoQ

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
# def genSamplePoints(geometry, gridScale, pointScale, seed):
#     projection = ee.Projection("EPSG:3310").atScale(gridScale)

#     baseGrid = ee.Image.random(seed, distribution="uniform").multiply(1e6).int()
#     pointsLayer = ee.Image.random(seed).multiply(1e6).int()
#     mask = ee.Image.pixelCoordinates(projection
#                   ).expression("!((b('x') + 0.5) % 2 != 0 || (b('y') + 0.5) % 2 != 0)")

#     grid = baseGrid.clip(geometry
#                   ).updateMask(mask
#                   ).reproject(projection)
    
#     gridMax = grid.addBands(pointsLayer
#                  ).reduceConnectedComponents(ee.Reducer.max())

#     points = pointsLayer.eq(gridMax
#                        ).selfMask(
#                        ).clip(geometry
#                        ).reproject(projection.scale(pointScale, pointScale))

#     # return grid, points
#     return points


# add variable tile scale based on fire size
# def reducePoints(featureCollection, name, pointsImage):
#     fireFeature = ee.Feature(featureCollection.filter(ee.Filter.eq("FIRE_NAME", name)).first())

#     points = pointsImage.reduceToVectors(reducer=ee.Reducer.countEvery(),
#                                          geometry=fireFeature.geometry(),
#                                          geometryType="centroid",
#                                          maxPixels=1e10).geometry().coordinates()
#     return points


def genSamplePoints(feature, gridScale, pointScale, seed):
    projection = ee.Projection("EPSG:3310").atScale(gridScale)
    geometry = feature.geometry()

    baseGrid = ee.Image.random(seed, distribution="uniform").multiply(1e6).int()
    pointsLayer = ee.Image.random(seed).multiply(1e6).int()
    mask = ee.Image.pixelCoordinates(projection
                  ).expression("!((b('x') + 0.5) % 2 != 0 || (b('y') + 0.5) % 2 != 0)")

    grid = baseGrid.clip(geometry
                  ).updateMask(mask
                  ).reproject(projection)
    
    gridMax = grid.addBands(pointsLayer
                 ).reduceConnectedComponents(ee.Reducer.max())

    points = pointsLayer.eq(gridMax
                       ).selfMask(
                       ).clip(geometry
                       ).reproject(projection.scale(pointScale, pointScale))

    return points.reduceToVectors(reducer=ee.Reducer.countEvery(),
                                  geometry=geometry,
                                  geometryType="centroid",
                                  maxPixels=1e10).geometry().coordinates()


# def reducePoints(geometry, pointsImage):
#     # fireFeature = ee.Feature(featureCollection.filter(ee.Filter.eq("FIRE_NAME", name)).first())

#     points = pointsImage.reduceToVectors(reducer=ee.Reducer.countEvery(),
#                                          geometry=fireFeature.geometry(),
#                                          geometryType="centroid",
#                                          maxPixels=1e10).geometry().coordinates()
#     return points


def formatToGPD(fireNames, pointLst, path):
    names = functools.reduce(operator.iconcat, [[fireNames[index]]*len(value) for index, value in enumerate(pointLst)], [])
    points = functools.reduce(operator.iconcat, pointLst, [])
    points_gpd = gpd.GeoSeries(map(Point, points))

    gdf = gpd.GeoDataFrame(names, geometry=points_gpd).rename(columns={0: "FIRE_NAME"})
    gdf.crs = "EPSG:4326"
    gdf.to_crs("EPSG:3310", inplace=True)
    # return gdf
    gdf.to_file(path)


# def savePoints(fireNames, coordinates, path):
#     gdf = gpd.GeoDataFrame(fireNames, geometry=coordinates).rename(columns={0: "FIRE_NAME"})
#     gdf.crs = "EPSG:4326"   # set crs for naive geometry
#     gdf.to_crs("EPSG:3310", inplace=True)
#     return gdf
#     # gdf.to_file(path)

In [ ]:
# Oscar's data can't be pushed to EE for some reason

# bbox_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Oscar/data/target_fires/unburned/target_fires_outlines.shp")

# # Change to burned area shp file
# fireBounds_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Oscar/data/target_fires/bounds/target_fires_bounds.shp")
# fireBounds_df["geometry"] = fireBounds_df["geometry"].difference(bbox_df["geometry"])

In [ ]:
bbox_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/bbox.shp")
fireBounds_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/fireBounds.shp")

In [ ]:
# push to EE as featureCollections
bbox_EE = geemap.gdf_to_ee(bbox_df, geodesic=True)
fireBounds_EE = geemap.gdf_to_ee(fireBounds_df, geodesic=True)

# Combined fire geometries
# fireGeometries = fireBounds_EE.geometry().dissolve()
# bboxGeometries = bbox_EE.geometry().dissolve()

In [ ]:
# checks if data is successfully pushed to EE
print(bbox_EE.size().getInfo(),
      fireBounds_EE.size().getInfo())

17 17


In [ ]:
# generate grid+points for fire and bbox geometries
# fireGrid, fireGridPoints = genSamplePoints(fireGeometries, 100, 1/8, 77)    # change grid scale for fires?
# bboxGrid, bboxGridPoints = genSamplePoints(bboxGeometries, 100, 1/8, 67)

In [ ]:
firePts, bboxPts = ee.List([]), ee.List([])

for name, size in fireBounds_df[["FIRE_NAME", "GIS_ACRES"]].values:
    if size < 40000:
        gridScale = 80
    elif size < 90000:
        gridScale = 100
    elif size < 150000:
        gridScale = 110
    else:
        gridScale = 125

    firePts = firePts.add(genSamplePoints(feature=fireBounds_EE.filter(ee.Filter.eq("FIRE_NAME", name)),
                                          gridScale=gridScale-10,
                                          pointScale=1/8,
                                          seed=random.randint(0, 1e6)))
    
    bboxPts = bboxPts.add(genSamplePoints(feature=bbox_EE.filter(ee.Filter.eq("FIRE_NAME", name)),
                                          gridScale=gridScale+10,
                                          pointScale=1/8,
                                          seed=random.randint(0, 1e6)))

In [ ]:
# 30mx30m

In [ ]:
startTime = time.time()
firePts = firePts.getInfo()
bboxPts = bboxPts.getInfo()

print((time.time()-startTime)/60)

4.419641828536987


In [ ]:
# formatToGPD(list(fireBounds_df["FIRE_NAME"]),
#             firePts,
#             "/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/sampleCoordinates.shp")    #120k  150k

# formatToGPD(list(fireBounds_df["FIRE_NAME"]),
#             bboxPts,
#             "/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleCoordinates.shp")    #199k  165k

# IGNORE BELOW

In [ ]:
print(cat.pivot_table(index="FIRE_NAME", values="geometry", aggfunc=len),
      dog.pivot_table(index="FIRE_NAME", values="geometry", aggfunc=len), sep="\n \n")

                geometry
FIRE_NAME               
ATLAS               6427
BALD                8220
BUCK                2759
BUTTE               8855
CALDWELL           10146
CAMP               11734
CARR               17572
CASCADE             3344
COVE                6376
FRYING PAN         13471
HAPPY               8496
KINCADE             9728
KING                9879
OAK                 9206
REDWOOD VALLEY      7536
ROCKY               8666
WALKER              6813
 
                geometry
FIRE_NAME               
ATLAS              10969
BALD                6212
BUCK                1934
BUTTE               8851
CALDWELL            9415
CAMP               13465
CARR               16686
CASCADE             3148
COVE                4243
FRYING PAN         17982
HAPPY               9395
KINCADE            11855
KING               20201
OAK                11923
REDWOOD VALLEY      5907
ROCKY               6416
WALKER              6459


In [ ]:
# # map of grids/points for all fires

# gridMap = geemap.Map()
# gridMap.centerObject(ee.Geometry.Point([40.145289, -121.890049][::-1]), 7)

# gridMap.addLayer(fireGrid, {"bands": ["random"], "palette": ["gray"]}, "fireGrid")
# gridMap.addLayer(bboxGrid, {"bands": ["random"], "palette": ["black"]}, "bboxGrid")

# gridMap.addLayer(fireGridPoints,
#                  {"bands": ["random"], "palette": ["red"]},
#                  "fireGridPoints")

# gridMap.addLayer(bboxGridPoints,
#                  {"bands": ["random"], "palette": ["white"]},
#                  "bboxGridPoints")

# gridMap

Map(center=[40.145289, -121.89004900000002], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [ ]:
# fireNames = list(fireBounds_df["FIRE_NAME"])
# fireNames_EE = ee.List(fireNames)

# startTime = time.time()

# # map reducer over fires to get coordinates 
# fireCoords = fireNames_EE.map(lambda x: reducePoints(fireBounds_EE, x, fireGridPoints)).getInfo()
# bboxCoords = fireNames_EE.map(lambda x: reducePoints(bbox_EE, x, bboxGridPoints)).getInfo()

# print(np.round((time.time()-startTime)/60, 3))

3.393


In [ ]:
# # array of corresponding fire names for coordinates
# names_1 = functools.reduce(operator.iconcat, [[fireNames[index]]*len(value) for index, value in enumerate(fireCoords)], [])
# names_2 = functools.reduce(operator.iconcat, [[fireNames[index]]*len(value) for index, value in enumerate(bboxCoords)], [])

# # unpack nested coordinates
# fireCoords = functools.reduce(operator.iconcat, fireCoords, [])
# bboxCoords = functools.reduce(operator.iconcat, bboxCoords, [])

# # coordinates to Point
# fireCoords = gpd.GeoSeries(map(Point, fireCoords))      #133k points with 100m grid spacing
# bboxCoords = gpd.GeoSeries(map(Point, bboxCoords))      #220K points with 100m grid spacing and 7.5% buffer

In [ ]:
# # save coordinates
# saveCoords(names_1, fireCoords, "/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/sampleCoordinates.shp")
# saveCoords(names_2, bboxCoords, "/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleCoordinates.shp")

## using coordinates

In [ ]:
# load coordinates
fireCoords_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/sampleCoordinates.shp")
bboxCoords_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleCoordinates.shp")

In [ ]:
print(fireCoords_df.pivot_table(index="FIRE_NAME", values="geometry", aggfunc=len),
      bboxCoords_df.pivot_table(index="FIRE_NAME", values="geometry", aggfunc=len), sep="\n \n")

                geometry
FIRE_NAME               
ATLAS               5230
BALD                4020
BUCK                1353
BUTTE               7159
CALDWELL            8220
CAMP               15504
CARR               23221
CASCADE             1633
COVE                3130
FRYING PAN         13471
HAPPY               6874
KINCADE             7869
KING                9858
OAK                 9219
REDWOOD VALLEY      3686
ROCKY               7031
WALKER              5505
 
                geometry
FIRE_NAME               
ATLAS              13303
BALD                5045
BUCK                1554
BUTTE              10703
CALDWELL           11398
CAMP               24486
CARR               30439
CASCADE             2555
COVE                3444
FRYING PAN         25886
HAPPY              11360
KINCADE            14332
KING               29099
OAK                17210
REDWOOD VALLEY      4768
ROCKY               7758
WALKER              7826


In [ ]:
# loading subset of coordinates in EE and visualize
atlasFirePoints = geemap.gdf_to_ee(fireCoords_df[fireCoords_df["FIRE_NAME"]=="ATLAS"])
atlasBBoxPoints = geemap.gdf_to_ee(bboxCoords_df[bboxCoords_df["FIRE_NAME"]=="ATLAS"])

atlasMap = geemap.Map()
atlasMap.centerObject(ee.Geometry.Point([40.145289, -121.890049][::-1]), 7)

atlasMap.addLayer(atlasFirePoints, {"bands": ["vis-red"], "palette": ["black"]}, "fire points")
atlasMap.addLayer(atlasBBoxPoints, {"bands": ["vis-red"], "palette": ["red"]}, "bbox points")

atlasMap

Map(center=[40.145289, -121.89004900000002], controls=(WidgetControl(options=['position', 'transparent_bg'], w…